In [6]:
from dotenv import load_dotenv
load_dotenv()

import os
import csv
import json
import time
import pandas as pd

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTAGE_API_KEY = os.environ.get('UPSTAGE_API_KEY')
LANGCHAIN_API_KEY = os.environ.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'matching_model_demo' # 프로젝트명 수정
LANGCHAIN_PROJECT = os.environ.get('LANGCHAIN_PROJECT')

print(f'LangSmith Project: {LANGCHAIN_PROJECT}')

LangSmith Project: matching_model_demo


# 데이터 구성

> csv to jsonl

In [3]:
text = pd.read_csv('../data/영문_텍스트.csv', dtype=str)
static = pd.read_csv('../data/통계청.csv', dtype=str)
customs = pd.read_csv('../data/관세청.csv', dtype=str)

## 영문 텍스트

In [4]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10000 non-null  object
 1   CODE    10000 non-null  object
 2   DSC     10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


In [5]:
text.loc[0]

ID                                                      1
CODE                                                 4520
DSC     automotive repair shops, nec  specialized auto...
Name: 0, dtype: object

- DSC는 page_content, 나머지는 metadata

In [8]:
csv.DictReader(text)

TypeError: 'DictReader' object is not subscriptable

## 통계청

## 관세청